In [5]:
use std::env;
use std::path::Path;

fn main() -> std::io::Result<()> {
    let path = env::current_dir()?;
    println!("The current directory is {}", path.display());
    Ok(())
}
main();

let path = Path::new("/home/bzhan/projects/postflop-solver");
assert!(env::set_current_dir(&path).is_ok());
println!("Successfully changed working directory to {}!", path.display());

The current directory is /home/bzhan/projects/postflop-solver
Successfully changed working directory to /home/bzhan/projects/postflop-solver!


In [2]:
:dep postflop-solver = { path = "/home/bzhan/projects/postflop-solver" }
use postflop_solver::*;
:dep polars = { version = "0.41.3" }
use polars::prelude::*;
use std::mem::MaybeUninit;
use std::slice;

## Game Manager tests

In [166]:
pub struct GameManager {
    game: PostFlopGame,
}

//#[inline]
fn decode_action(action: &str) -> Action {
    match action {
        "F" => Action::Fold,
        "X" => Action::Check,
        "C" => Action::Call,
        _ => {
            let mut chars = action.chars();
            let first_char = chars.next().unwrap();
            let amount = chars.as_str().parse().unwrap();
            match first_char {
                'B' => Action::Bet(amount),
                'R' => Action::Raise(amount),
                'A' => Action::AllIn(amount),
                _ => unreachable!(),
            }
        }
    }
}

//#[inline]
fn round(value: f64) -> f64 {
    if value < 1.0 {
        (value * 1000000.0).round() / 1000000.0
    } else if value < 10.0 {
        (value * 100000.0).round() / 100000.0
    } else if value < 100.0 {
        (value * 10000.0).round() / 10000.0
    } else if value < 1000.0 {
        (value * 1000.0).round() / 1000.0
    } else if value < 10000.0 {
        (value * 100.0).round() / 100.0
    } else {
        (value * 10.0).round() / 10.0
    }
}

//#[inline]
fn round_iter<'a>(iter: impl Iterator<Item = &'a f32> + 'a) -> impl Iterator<Item = f64> + 'a {
    iter.map(|&x| round(x as f64))
}

//#[inline]
pub fn weighted_average(slice: &[f32], weights: &[f32]) -> f64 {
    let mut sum = 0.0;
    let mut weight_sum = 0.0;
    for (&value, &weight) in slice.iter().zip(weights.iter()) {
        sum += value as f64 * weight as f64;
        weight_sum += weight as f64;
    }
    sum / weight_sum
}

//#[wasm_bindgen]
impl GameManager {
    pub fn new() -> Self {
        Self {
            game: PostFlopGame::new(),
        }
    }

    pub fn init(
        &mut self,
        oop_range: &[f32],
        ip_range: &[f32],
        board: &[u8],
        starting_pot: i32,
        effective_stack: i32,
        rake_rate: f64,
        rake_cap: f64,
        donk_option: bool,
        oop_flop_bet: &str,
        oop_flop_raise: &str,
        oop_turn_bet: &str,
        oop_turn_raise: &str,
        oop_turn_donk: &str,
        oop_river_bet: &str,
        oop_river_raise: &str,
        oop_river_donk: &str,
        ip_flop_bet: &str,
        ip_flop_raise: &str,
        ip_turn_bet: &str,
        ip_turn_raise: &str,
        ip_river_bet: &str,
        ip_river_raise: &str,
        add_allin_threshold: f64,
        force_allin_threshold: f64,
        merging_threshold: f64,
        added_lines: &str,
        removed_lines: &str,
    ) -> Option<String> {
        let (turn, river, state) = match board.len() {
            3 => (NOT_DEALT, NOT_DEALT, BoardState::Flop),
            4 => (board[3], NOT_DEALT, BoardState::Turn),
            5 => (board[3], board[4], BoardState::River),
            _ => return Some("Invalid board length".to_string()),
        };

        let card_config = CardConfig {
            range: [
                Range::from_raw_data(oop_range).unwrap(),
                Range::from_raw_data(ip_range).unwrap(),
            ],
            flop: board[..3].try_into().unwrap(),
            turn,
            river,
        };

        let tree_config = TreeConfig {
            initial_state: state,
            starting_pot,
            effective_stack,
            rake_rate,
            rake_cap,
            flop_bet_sizes: [
                BetSizeOptions::try_from((oop_flop_bet, oop_flop_raise)).unwrap(),
                BetSizeOptions::try_from((ip_flop_bet, ip_flop_raise)).unwrap(),
            ],
            turn_bet_sizes: [
                BetSizeOptions::try_from((oop_turn_bet, oop_turn_raise)).unwrap(),
                BetSizeOptions::try_from((ip_turn_bet, ip_turn_raise)).unwrap(),
            ],
            river_bet_sizes: [
                BetSizeOptions::try_from((oop_river_bet, oop_river_raise)).unwrap(),
                BetSizeOptions::try_from((ip_river_bet, ip_river_raise)).unwrap(),
            ],
            turn_donk_sizes: match donk_option {
                false => None,
                true => DonkSizeOptions::try_from(oop_turn_donk).ok(),
            },
            river_donk_sizes: match donk_option {
                false => None,
                true => DonkSizeOptions::try_from(oop_river_donk).ok(),
            },
            add_allin_threshold,
            force_allin_threshold,
            merging_threshold,
        };

        let mut action_tree = ActionTree::new(tree_config).unwrap();

        if !added_lines.is_empty() {
            for added_line in added_lines.split(',') {
                let line = added_line
                    .split(&['-', '|'][..])
                    .map(decode_action)
                    .collect::<Vec<_>>();
                if action_tree.add_line(&line).is_err() {
                    return Some("Failed to add line (loaded broken tree?)".to_string());
                }
            }
        }

        if !removed_lines.is_empty() {
            for removed_line in removed_lines.split(',') {
                let line = removed_line
                    .split(&['-', '|'][..])
                    .map(decode_action)
                    .collect::<Vec<_>>();
                if action_tree.remove_line(&line).is_err() {
                    return Some("Failed to remove line (loaded broken tree?)".to_string());
                }
            }
        }

        self.game.update_config(card_config, action_tree).err()
    }

    pub fn private_cards(&self, player: usize) -> Box<[u16]> {
        let cards = self.game.private_cards(player);
        cards
            .iter()
            .map(|&(c1, c2)| c1 as u16 | ((c2 as u16) << 8))
            .collect()
    }

    pub fn memory_usage(&self, enable_compression: bool) -> u64 {
        if !enable_compression {
            self.game.memory_usage().0
        } else {
            self.game.memory_usage().1
        }
    }

    pub fn allocate_memory(&mut self, enable_compression: bool) {
        self.game.allocate_memory(enable_compression);
    }

    pub fn solve_step(&self, current_iteration: u32) {
        unsafe {

            solve_step(&self.game, current_iteration);

        }
    }

    pub fn exploitability(&self) -> f32 {
        unsafe {
            compute_exploitability(&self.game)

        }
    }

    pub fn finalize(&mut self) {
        unsafe {
            
            finalize(&mut self.game);
            
        }
    }

    pub fn apply_history(&mut self, history: &[usize]) {
        self.game.apply_history(history);
    }

    pub fn total_bet_amount(&mut self, append: &[usize]) -> Box<[u32]> {
        if append.is_empty() {
            let total_bet_amount = self.game.total_bet_amount();
            return total_bet_amount.iter().map(|&x| x as u32).collect();
        }
        let history = self.game.history().to_vec();
        for &action in append {
            self.game.play(action);
        }
        let total_bet_amount = self.game.total_bet_amount();
        let ret = total_bet_amount.iter().map(|&x| x as u32).collect();
        self.game.apply_history(&history);
        ret
    }

    pub fn current_player(&self) -> String {
        if self.game.is_terminal_node() {
            "terminal".to_string()
        } else if self.game.is_chance_node() {
            "chance".to_string()
        } else if self.game.current_player() == 0 {
            "oop".to_string()
        } else {
            "ip".to_string()
        }
    }

    pub fn num_actions(&self) -> usize {
        self.game.available_actions().len()
    }

    fn actions(&self) -> String {
        if self.game.is_terminal_node() {
            "terminal".to_string()
        } else if self.game.is_chance_node() {
            "chance".to_string()
        } else {
            self.game
                .available_actions()
                .iter()
                .map(|&x| match x {
                    Action::Fold => "Fold:0".to_string(),
                    Action::Check => "Check:0".to_string(),
                    Action::Call => "Call:0".to_string(),
                    Action::Bet(amount) => format!("Bet:{amount}"),
                    Action::Raise(amount) => format!("Raise:{amount}"),
                    Action::AllIn(amount) => format!("Allin:{amount}"),
                    _ => unreachable!(),
                })
                .collect::<Vec<_>>()
                .join("/")
        }
    }

    pub fn actions_after(&mut self, append: &[usize]) -> String {
        if append.is_empty() {
            return self.actions();
        }
        let history = self.game.history().to_vec();
        for &action in append {
            self.game.play(action);
        }
        let ret = self.actions();
        self.game.apply_history(&history);
        ret
    }

    pub fn possible_cards(&self) -> u64 {
        self.game.possible_cards()
    }

    pub fn get_results(&mut self) -> Box<[f64]> {
        let game = &mut self.game;
        let mut buf = Vec::new();

        let total_bet_amount = game.total_bet_amount();
        let pot_base = game.tree_config().starting_pot + total_bet_amount.iter().min().unwrap();

        buf.push((pot_base + total_bet_amount[0]) as f64);
        buf.push((pot_base + total_bet_amount[1]) as f64);

        let trunc = |&w: &f32| if w < 0.0005 { 0.0 } else { w };
        let weights = [
            game.weights(0).iter().map(trunc).collect::<Vec<_>>(),
            game.weights(1).iter().map(trunc).collect::<Vec<_>>(),
        ];

        let is_empty = |player: usize| weights[player].iter().all(|&w| w == 0.0);
        let is_empty_flag = is_empty(0) as usize + 2 * is_empty(1) as usize;
        buf.push(is_empty_flag as f64);

        buf.extend(round_iter(weights[0].iter()));

        buf.extend(round_iter(weights[1].iter()));

        if is_empty_flag > 0 {
            buf.extend(round_iter(weights[0].iter()));
            buf.extend(round_iter(weights[1].iter()));
        } else {
            game.cache_normalized_weights();

            buf.extend(round_iter(game.normalized_weights(0).iter()));
            buf.extend(round_iter(game.normalized_weights(1).iter()));

            let equity = [game.equity(0), game.equity(1)];
            let ev = [game.expected_values(0), game.expected_values(1)];

            buf.extend(round_iter(equity[0].iter()));
            buf.extend(round_iter(equity[1].iter()));
            buf.extend(round_iter(ev[0].iter()));
            buf.extend(round_iter(ev[1].iter()));

            for player in 0..2 {
                let pot = (pot_base + total_bet_amount[player]) as f64;
                for (&eq, &ev) in equity[player].iter().zip(ev[player].iter()) {
                    let (eq, ev) = (eq as f64, ev as f64);
                    if eq < 5e-7 {
                        buf.push(ev / 0.0);
                    } else {
                        buf.push(round(ev / (pot * eq)));
                    }
                }
            }
        }

        if !game.is_terminal_node() && !game.is_chance_node() {
            buf.extend(round_iter(game.strategy().iter()));
            if is_empty_flag == 0 {
                buf.extend(round_iter(
                    game.expected_values_detail(game.current_player()).iter(),
                ));
            }
        }

        buf.into_boxed_slice()
    }

    pub fn get_chance_reports(&mut self, append: &[usize], num_actions: usize) -> Box<[f64]> {
        let game = &mut self.game;
        let history = game.history().to_vec();

        let mut status = vec![0.0; 52]; // 0: not possible, 1: empty, 2: not empty
        let mut combos = [vec![0.0; 52], vec![0.0; 52]];
        let mut equity = [vec![0.0; 52], vec![0.0; 52]];
        let mut ev = [vec![0.0; 52], vec![0.0; 52]];
        let mut eqr = [vec![0.0; 52], vec![0.0; 52]];
        let mut strategy = vec![0.0; num_actions * 52];

        let possible_cards = game.possible_cards();
        for chance in 0..52 {
            if possible_cards & (1 << chance) == 0 {
                continue;
            }

            game.play(chance);
            for &action in &append[1..] {
                game.play(action);
            }

            let trunc = |&w: &f32| if w < 0.0005 { 0.0 } else { w };
            let weights = [
                game.weights(0).iter().map(trunc).collect::<Vec<_>>(),
                game.weights(1).iter().map(trunc).collect::<Vec<_>>(),
            ];

            combos[0][chance] = round(weights[0].iter().fold(0.0, |acc, &w| acc + w as f64));
            combos[1][chance] = round(weights[1].iter().fold(0.0, |acc, &w| acc + w as f64));

            let is_empty = |player: usize| weights[player].iter().all(|&w| w == 0.0);
            let is_empty_flag = [is_empty(0), is_empty(1)];

            game.cache_normalized_weights();
            let normalizer = [game.normalized_weights(0), game.normalized_weights(1)];

            if !game.is_terminal_node() {
                let current_player = game.current_player();
                if !is_empty_flag[current_player] {
                    let strategy_tmp = game.strategy();
                    let num_hands = game.private_cards(current_player).len();
                    let ws = if is_empty_flag[current_player ^ 1] {
                        &weights[current_player]
                    } else {
                        normalizer[current_player]
                    };
                    for action in 0..num_actions {
                        let slice = &strategy_tmp[action * num_hands..(action + 1) * num_hands];
                        let strategy_summary = weighted_average(slice, ws);
                        strategy[action * 52 + chance] = round(strategy_summary);
                    }
                }
            }

            if is_empty_flag[0] || is_empty_flag[1] {
                status[chance] = 1.0;
                game.apply_history(&history);
                continue;
            }

            status[chance] = 2.0;

            let total_bet_amount = game.total_bet_amount();
            let pot_base = game.tree_config().starting_pot + total_bet_amount.iter().min().unwrap();

            for player in 0..2 {
                let pot = (pot_base + total_bet_amount[player]) as f32;
                let equity_tmp = weighted_average(&game.equity(player), normalizer[player]);
                let ev_tmp = weighted_average(&game.expected_values(player), normalizer[player]);
                equity[player][chance] = round(equity_tmp);
                ev[player][chance] = round(ev_tmp);
                eqr[player][chance] = round(ev_tmp / (pot as f64 * equity_tmp));
            }

            game.apply_history(&history);
        }

        let mut buf = Vec::new();

        buf.extend_from_slice(&status);
        buf.extend_from_slice(&combos[0]);
        buf.extend_from_slice(&combos[1]);
        buf.extend_from_slice(&equity[0]);
        buf.extend_from_slice(&equity[1]);
        buf.extend_from_slice(&ev[0]);
        buf.extend_from_slice(&ev[1]);
        buf.extend_from_slice(&eqr[0]);
        buf.extend_from_slice(&eqr[1]);
        buf.extend_from_slice(&strategy);

        buf.into_boxed_slice()
    }
}

In [170]:
let mut gm = GameManager{game: temp_game};

In [175]:
let strat = gm.game.strategy();

In [183]:
(&(gm.game).private_cards(0)).len()

167

In [187]:
gm.game.available_actions()

[Check, Bet(120), Bet(216), AllIn(900)]

In [171]:
gm.get_results()

[200.0, 200.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 

## Game tests

In [11]:
use polars::prelude::*;
use std::collections::HashMap;

fn main() -> PostFlopGame { //Result<(DataFrame, DataFrame)> {
    // ranges of OOP and IP in string format
    // see the documentation of `Range` for more details about the format
    let oop_range = "66+,A8s+,A5s-A4s,AJo+,K9s+,KQo,QTs+,JTs,96s+,85s+,75s+,65s,54s";
    let ip_range = "QQ-22,AQs-A2s,ATo+,K5s+,KJo+,Q8s+,J8s+,T7s+,96s+,86s+,75s+,64s+,53s+";

    let card_config = CardConfig {
        range: [oop_range.parse().unwrap(), ip_range.parse().unwrap()],
        flop: flop_from_str("Td9d6h").unwrap(),
        turn: card_from_str("Qc").unwrap(),
        river: NOT_DEALT,
    };

    // bet sizes -> 60% of the pot, geometric size, and all-in
    // raise sizes -> 2.5x of the previous bet
    // see the documentation of `BetSizeOptions` for more details
    let bet_sizes = BetSizeOptions::try_from(("60%, e, a", "2.5x")).unwrap();

    let tree_config = TreeConfig {
        initial_state: BoardState::Turn, // must match `card_config`
        starting_pot: 200,
        effective_stack: 900,
        rake_rate: 0.0,
        rake_cap: 0.0,
        flop_bet_sizes: [bet_sizes.clone(), bet_sizes.clone()], // [OOP, IP]
        turn_bet_sizes: [bet_sizes.clone(), bet_sizes.clone()],
        river_bet_sizes: [bet_sizes.clone(), bet_sizes],
        turn_donk_sizes: None, // use default bet sizes
        river_donk_sizes: Some(DonkSizeOptions::try_from("50%").unwrap()),
        add_allin_threshold: 1.5, // add all-in if (maximum bet size) <= 1.5x pot
        force_allin_threshold: 0.15, // force all-in if (SPR after the opponent's call) <= 0.15
        merging_threshold: 0.1,
    };

    // build the game tree
    // `ActionTree` can be edited manually after construction
    let action_tree = ActionTree::new(tree_config).unwrap();
    let mut game = PostFlopGame::with_config(card_config, action_tree).unwrap();

    // obtain the private hands
    let oop_cards: &[(u8, u8)] = game.private_cards(0);
    let oop_cards_str = holes_to_strings(oop_cards).unwrap();
    println!("{:?}", oop_cards_str);

    let ip_cards = game.private_cards(1);
    let ip_cards_str = holes_to_strings(ip_cards).unwrap();
    println!("{:?}", ip_cards_str);
    
    let (mem_usage, mem_usage_compressed) = game.memory_usage();
    println!(
        "Memory usage without compression (32-bit float): {:.2}GB",
        mem_usage as f64 / (1024.0 * 1024.0 * 1024.0)
    );
    println!(
        "Memory usage with compression (16-bit integer): {:.2}GB",
        mem_usage_compressed as f64 / (1024.0 * 1024.0 * 1024.0)
    );

    game.allocate_memory(false);

    let max_num_iterations = 1000;
    let target_exploitability = game.tree_config().starting_pot as f32 * 0.005; // 0.5% of the pot
    println!("Target exploitability: {:.2}", target_exploitability);
    //let exploitability = solve(&mut game, max_num_iterations, target_exploitability, true);
    //println!("Exploitability: {:.2}", exploitability);
    
    let mut last_exploit = 1000.0;
    for i in 0..max_num_iterations {
        solve_step(&game, i);
        if (i + 1) % 100 == 0 {
            let exploitability = compute_exploitability(&game);
            println!("Exploitability on iteration {}: {:.2}", i, exploitability);
            if ((last_exploit - exploitability) < 0.01) {
                break
            }
            last_exploit=exploitability;
        }
     }
    finalize(&mut game);
    game.cache_normalized_weights();
    game
    }

In [12]:
let game = main();

["5c4c", "Ac4c", "5d4d", "Ad4d", "5h4h", "Ah4h", "5s4s", "As4s", "6c5c", "7c5c", "8c5c", "Ac5c", "6d5d", "7d5d", "8d5d", "Ad5d", "7h5h", "8h5h", "Ah5h", "6s5s", "7s5s", "8s5s", "As5s", "6d6c", "6s6c", "7c6c", "8c6c", "9c6c", "6s6d", "7d6d", "8d6d", "7s6s", "8s6s", "9s6s", "7d7c", "7h7c", "7s7c", "8c7c", "9c7c", "7h7d", "7s7d", "8d7d", "7s7h", "8h7h", "9h7h", "8s7s", "9s7s", "8d8c", "8h8c", "8s8c", "9c8c", "Ac8c", "8h8d", "8s8d", "Ad8d", "8s8h", "9h8h", "Ah8h", "9s8s", "As8s", "9h9c", "9s9c", "Kc9c", "Ac9c", "9s9h", "Kh9h", "Ah9h", "Ks9s", "As9s", "ThTc", "TsTc", "JcTc", "KcTc", "AcTc", "TsTh", "JhTh", "QhTh", "KhTh", "AhTh", "JsTs", "QsTs", "KsTs", "AsTs", "JdJc", "JhJc", "JsJc", "KcJc", "AcJc", "AdJc", "AhJc", "AsJc", "JhJd", "JsJd", "QdJd", "KdJd", "AcJd", "AdJd", "AhJd", "AsJd", "JsJh", "QhJh", "KhJh", "AcJh", "AdJh", "AhJh", "AsJh", "QsJs", "KsJs", "AcJs", "AdJs", "AhJs", "AsJs", "QhQd", "QsQd", "KcQd", "KdQd", "KhQd", "KsQd", "AcQd", "AdQd", "AhQd", "AsQd", "QsQh", "KcQh", "KdQh",

In [55]:
use std::error::Error;

fn get_summary_results(game: &PostFlopGame, player: usize) -> Result<DataFrame> {
    
    let cards: &[(u8, u8)] = game.private_cards(player);
    let cards = holes_to_strings(cards).unwrap();
    println!("{}", cards.len());
    println!("{}", &cards[0..3]);

    let df = DataFrame::new(vec![Series::new("cards", &cards),
                                Series::new("equities", game.equity(player)),
                                Series::new("ev", game.expected_values(player)),
                                ])?;
    let mut df = df.clone();                            
    
    // Add action proportions of each card
    let card_len = cards.len();
    let strat = game.strategy();
    let available_actions = game.available_actions();
    let mut counter = 0;
    for (index, action) in available_actions.iter().enumerate() {
        println!("{}", &action.to_string());
        let new_series = Series::new(&action.to_string(), &strat[counter..(counter + card_len)]);
        df = df.hstack(&[new_series])?;
        //let df_head = &df.head(Some(3));
        //println!("{}", df_head);
        counter += card_len;
    }

    Ok(df)
    }

In [ ]:
let df = get_summary_results(&game, 0).unwrap();

In [53]:
df

shape: (167, 7)
┌───────┬──────────┬────────────┬──────────┬──────────┬──────────┬─────────────┐
│ cards ┆ equities ┆ ev         ┆ Check    ┆ Bet(120) ┆ Bet(216) ┆ All-in(900) │
│ ---   ┆ ---      ┆ ---        ┆ ---      ┆ ---      ┆ ---      ┆ ---         │
│ str   ┆ f32      ┆ f32        ┆ f32      ┆ f32      ┆ f32      ┆ f32         │
╞═══════╪══════════╪════════════╪══════════╪══════════╪══════════╪═════════════╡
│ 5c4c  ┆ 0.052051 ┆ 7.292892   ┆ 1.0      ┆ 0.0      ┆ 0.0      ┆ 0.0         │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Ac4c  ┆ 0.145191 ┆ 9.656242   ┆ 1.0      ┆ 0.0      ┆ 0.0      ┆ 0.0         │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 5d4d  ┆ 0.233773 ┆ 37.935509  ┆ 1.0      ┆ 0.0      ┆ 0.0      ┆ 0.0         │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ Ad4d  ┆ 0.326162 ┆ 49.981922  ┆ 0.715881 ┆ 0.284106 ┆ 0.000014 ┆ 0.0         │
├╌╌╌╌╌╌╌┼╌╌╌

In [57]:
game.available_actions()

[Check, Bet(120), Bet(216), AllIn(900)]

In [62]:
//onto the IP player after OOP chooses bet(120)
let mut game = game;
game.play(1)
game.cache_normalized_weights()

()

In [ ]:
let df2 = get_summary_results(&game, 1).unwrap();

In [6]:
let current_board_strings: Vec<String> = game.current_board().iter().map(|&x| card_to_string(x).unwrap()).collect();
current_board_strings

Error: cannot find value `game` in this scope

In [102]:
df2.sort(["ev"], true)?

shape: (250, 6)
┌───────┬──────────┬────────────┬──────────┬──────────┬────────────┐
│ cards ┆ equities ┆ ev         ┆ Fold     ┆ Call     ┆ Raise(300) │
│ ---   ┆ ---      ┆ ---        ┆ ---      ┆ ---      ┆ ---        │
│ str   ┆ f32      ┆ f32        ┆ f32      ┆ f32      ┆ f32        │
╞═══════╪══════════╪════════════╪══════════╪══════════╪════════════╡
│ KdJd  ┆ 0.890995 ┆ 523.942322 ┆ 0.0      ┆ 0.060269 ┆ 0.939731   │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
│ KdJc  ┆ 0.861111 ┆ 474.342163 ┆ 0.0      ┆ 0.315794 ┆ 0.684206   │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
│ KdJh  ┆ 0.860514 ┆ 473.103271 ┆ 0.0      ┆ 0.319202 ┆ 0.680798   │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
│ KdJs  ┆ 0.8606   ┆ 472.603088 ┆ 0.0      ┆ 0.364926 ┆ 0.635074   │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┤
│ ...   ┆ ...      ┆ ...        ┆ ...      ┆ ...      ┆ ...        │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌

In [111]:
let col_expressions: Vec<Expr> = game.available_actions().into_iter().map(|col_name| col(col_name)).collect();

Error: cannot find type `Expr` in this scope

Error: unnecessary parentheses around `if` condition

Error: cannot find function `col` in this scope

Error: unused variable: `index`

In [104]:
df.lazy().select([
            col("ev"), 
            col("B"), 
            col("C"),
        ])
        .collect()?;

Error: unnecessary parentheses around `if` condition

Error: cannot find function `sum` in this scope

Error: unused variable: `index`